# Get the fidelities and runtimes for the repetition codes

Long runtime should be anticipated, and vary for different machine used. 

In [1]:
using Distributed

num_cores = length(Sys.cpu_info())
if nprocs()==1
    addprocs(num_cores; exeflags=`--project=$(Base.active_project())`)
end    

@everywhere begin
    using LatticeAlgorithms
    using Counters
end
using Plots
using JLD2

In [10]:
num_samples = 1e6
σrange = range(0.4, 0.8, 50)
Nmin = 1
Nmax = 30
Nrange = Nmin:Nmax

1:30

## Rep_rec_N code

In [11]:
datalist = pmap(σrange) do σ
    p0range_temp = []
    times_temp = []
    for N in Nrange
        M = rep_rec(N)
        Mperp = GKP_logical_operator_generator_canonical(M)
        ξs = [σ * randn(2N) for i in 1:num_samples]
        time = @elapsed ys = [decode_rep_rec(ξ) for ξ in ξs]
        us = [inv(transpose(√(2π) * Mperp)) * y for y in ys]
        logicals = [mod.(round.(Int, u[1:2]), 2) for u in us]
        push!(p0range_temp, counter(logicals)[[0,0]]/num_samples)
        push!(times_temp, time/num_samples)            
    end
    return p0range_temp, times_temp
end;

In [12]:
# # Uncomment this cell if you want to overwrite the data

# fn = "data/repetition_codes/fidelity_time_rep_rec_$(Nmin)_$(Nmax).jld2";
# jldsave(fn; 
#     σrange=σrange, 
#     Nrange=Nrange, 
#     datalist=datalist,
#     num_samples=num_samples
# )

## YY_rep_rec_N code

In [ ]:
num_samples = 1e6
σrange = range(0.4, 0.8, 50)
Nmin = 1
Nmax = 20
Nrange = Nmin:Nmax

In [ ]:
datalist = pmap(σrange) do σ
    p0range_temp = []
    times_temp = []                
    for N in Nrange
        M = YY_rep_rec(N)
        Mperp = GKP_logical_operator_generator_canonical(M)
        _, _, Q2, r2 = tlq_YY_rep_rec(N)
        ξs = [σ * randn(4N) for i in 1:num_samples]
        time = @elapsed ys = [decode_YY_rep_rec(ξ, Q2, r2) for ξ in ξs]
        us = [inv(transpose(√(2π) * Mperp)) * y for y in ys]
        logicals = [mod.(round.(Int, u[1:2]), 2) for u in us]
        push!(p0range_temp, counter(logicals)[[0,0]]/num_samples)
        push!(times_temp, time/num_samples)            
    end
    return p0range_temp, times_temp
end;

In [ ]:
# # Uncomment this cell if you want to overwrite the data

# fn = "data/repetition_codes/fidelity_time_YY_rep_rec_$(Nmin)_$(Nmax).jld2";
# jldsave(fn; 
#     σrange=σrange, 
#     Nrange=Nrange, 
#     datalist=datalist,
#     num_samples=num_samples
# )

# Get the runtime with exponential time decoder

## Rep_rec code

In [ ]:
num_samples = 1e6
Nrange = 1 : 10

times = pmap(Nrange) do N
    M = rep_rec(N)
    Mperp = GKP_logical_operator_generator(M)
    ξs = [0.6 * randn(2N) for i in 1:num_samples]
    
    t1 = @elapsed ys1 = [decode_rep_rec(ξ) for ξ in ξs]
    t2 = @elapsed ys2 = [closest_point(ξ, √(2π) * Mperp) for ξ in ξs]    
    
    @assert ys1 ≈ ys2
    
    return t1/num_samples, t2/num_samples
end

time_linear = [t[1] for t in times]
time_exp = [t[2] for t in times] ;


In [19]:
# # Uncomment this cell if you want to overwrite the data

# fn = "data/repetition_codes/runtime_rep_rec.jld2";
# jldsave(fn; 
#     Nrange=Nrange, 
#     time_exp=time_exp, 
#     time_linear=time_linear,     
#     num_samples=num_samples)

## YY-rep-rec code

In [ ]:
num_samples = 1e6
Nrange = 1 : 10

times = pmap(Nrange) do N
    M = YY_rep_rec(N)
    Mperp = GKP_logical_operator_generator(M)
    ξs = [0.6 * randn(2N) for i in 1:num_samples]
    
    t1 = @elapsed ys1 = [decode_YY_rep_rec(ξ) for ξ in ξs]
    t2 = @elapsed ys2 = [closest_point(ξ, √(2π) * Mperp) for ξ in ξs]    
    
    @assert ys1 ≈ ys2
    
    return t1/num_samples, t2/num_samples
end

time_linear = [t[1] for t in times]
time_exp = [t[2] for t in times] ;


In [ ]:
# # Uncomment this cell if you want to overwrite the data

# fn = "data/repetition_codes/runtime_YY_rep_rec.jld2";
# jldsave(fn; 
#     Nrange=Nrange, 
#     time_exp=time_exp, 
#     time_linear=time_linear,     
#     num_samples=num_samples)